In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

In [ ]:
N = 2048*2*2
fs = 192e3
t = np.arange(N) / fs
ftone = 1000
if False:
    i = 0.2*np.cos(2*np.pi*(ftone)*t)+np.random.randn(N)/300
    q = 0.2*np.sin(2*np.pi*(ftone)*t)+np.random.randn(N)/300
    si = (i*2048+1024).astype(int)
    sq = (q*2048+1024).astype(int)
    fle = "mock_L_data_int_1khz.c"
    fout = open(fle,'w')
    fout.write("int16_t L_mock_1khz[%d] = {"%N)
    for k in range(N-1):
        fout.write("%d,"%si[k]);
    fout.write("%d};\n"%(si[N-1]))
    fout.close()

    fle = "mock_R_data_int_1khz.c"
    fout = open(fle,'w')
    fout.write("int16_t R_mock_1khz[%d] = {"%N)
    for k in range(N-1):
        fout.write("%d,"%(sq[k]));
    fout.write("%d};\n"%(sq[N-1]))
    fout.close()
else:
    L = open('mock_L_data_int_1khz.c','r').readline()
    Lvals = L.split('=')[1].strip().strip(';{}').split(',')
    si = np.array([int(a) for a in Lvals])
    
    R = open('mock_R_data_int_1khz.c','r').readline()
    Rvals = R.split('=')[1].strip().strip(';{}').split(',')
    sq = np.array([int(a) for a in Rvals])
    
plt.figure()
plt.plot(t*1000,si,'k-',label='L/I')
plt.plot(t*1000,sq,'r-',label='R/Q')
plt.legend()
plt.xlabel("Time [ms]")
plt.ylabel('Digital units')

In [ ]:
N = 2048*2*2
fs = 192e3
t = np.arange(N) / fs
ftone = 440
if False:
    i = 0.9*0.1*np.cos(2*np.pi*(-48000-ftone)*t)+np.random.randn(N)/300
    q = 0.1*np.sin(2*np.pi*(-48000-ftone)*t+5*np.pi/180)+np.random.randn(N)/300
    si = (i*2048+1024).astype(int)
    sq = (q*2048+1024).astype(int)
    fle = "mock_L_data_int.c"
    fout = open(fle,'w')
    fout.write("int16_t L_mock[%d] = {"%N)
    for k in range(N-1):
        fout.write("%d,"%si[k]);
    fout.write("%d};\n"%(si[N-1]))
    fout.close()
    
    fle = "mock_R_data_int.c"
    fout = open(fle,'w')
    fout.write("int16_t R_mock[%d] = {"%N)
    for k in range(N-1):
        fout.write("%d,"%(sq[k]));
    fout.write("%d};\n"%(sq[N-1]))
    fout.close()
else:
    L = open('mock_L_data_int.c','r').readline()
    Lvals = L.split('=')[1].strip().strip(';{}').split(',')
    si = np.array([int(a) for a in Lvals])
    
    R = open('mock_R_data_int.c','r').readline()
    Rvals = R.split('=')[1].strip().strip(';{}').split(',')
    sq = np.array([int(a) for a in Rvals])

plt.figure()
plt.plot(t*1000,si,'k-',label='L/I')
plt.plot(t*1000,sq,'r-',label='R/Q')
plt.legend()
plt.xlabel("Time [ms]")
plt.ylabel('Digital units')

In [ ]:
data = si + 1j*sq
f,Pxx = welch(data,fs=fs,return_onesided=False)
plt.figure()
plt.plot(f/1000,10*np.log10(Pxx),'k.')
plt.xlabel('Frequency [kHz]')
plt.ylabel('Power [dB]')

In [ ]:
def phase_correction(I,Q,phs):
    if phs < 0:
        corr = phs*I
        Qcorr = Q + corr
        return I,Qcorr
    else:
        corr = phs*Q
        Icorr = I + corr
        return Icorr,Q

def apply_correction(I,Q,amp,phs):
    Icorr = I*amp
    return phase_correction(Icorr,Q,phs)

In [ ]:
Icorr,Qcorr = apply_correction(si/32768,sq/32768,1.11,-0.09)
dataC = Icorr+1j*Qcorr
fc,Pc = welch(dataC,fs=fs,return_onesided=False)
plt.figure()
plt.plot(fc/1000,10*np.log10(Pc),'k.')
plt.xlabel('Frequency [kHz]')
plt.ylabel('Power [dB]')
plt.grid()

In [ ]:
fle = "mock_I_data_IQcorrected.c"
fout = open(fle,'w')
fout.write("float32_t I_corrected[%d] = {"%N)
for k in range(N-1):
    fout.write("%7.6f,"%Icorr[k]);
fout.write("%7.6f};\n"%(Icorr[N-1]))
fout.close()

fle = "mock_Q_data_IQcorrected.c"
fout = open(fle,'w')
fout.write("float32_t Q_corrected[%d] = {"%N)
for k in range(N-1):
    fout.write("%7.6f,"%(Qcorr[k]));
fout.write("%7.6f};\n"%(Qcorr[N-1]))
fout.close()

In [ ]:
I = np.array([+1, 0,-1, 0]*128)
Q = np.array([ 0,-1, 0,+1]*128)
d = I + 1j*Q
D = np.fft.fft(d)
print("%d,%f"%(np.argmax(np.real(D)),np.abs(D[384])))

In [ ]:
I = np.array([+1, 0,-1, 0]*128)
Q = np.array([ 0,-1, 0,+1]*128)
d = I + 1j*Q
D = np.fft.fft(d)*np.hanning(512)
print("%d,%f"%(np.argmax(np.real(D)),np.abs(D[384])))

# Synchronous AM Algorithm Development

In [ ]:
# First, generate an AM signal
N = 256*32*2
fs = 24e3
t = np.arange(N) / fs
ftone = 400
fcarrier = 12
mod_index = 0.1

if False:
    # Modulating signal (1 kHz sine wave)
    message = np.sin(2 * np.pi * ftone * t)

    # AM signal: carrier * (1 + mod_index * message)
    # For 100% modulation, set mod_index = 1.0
    am_signal = (1 + mod_index * message) * np.cos(2 * np.pi * fcarrier * t)

    I = (1 + mod_index * message) * np.cos(2 * np.pi * fcarrier * t)
    Q = (1 + mod_index * message) * np.sin(2 * np.pi * fcarrier * t)

    si = (I*2048+1024).astype(int)
    sq = (Q*2048+1024).astype(int)
    
    fle = "mock_L_data_SAM.c"
    fout = open(fle,'w')
    fout.write("float32_t L_mock_SAM[%d] = {"%N)
    for k in range(N-1):
        fout.write("%6.5f,"%I[k]);
    fout.write("%6.5f};\n"%(I[N-1]))
    fout.close()
    
    fle = "mock_R_data_SAM.c"
    fout = open(fle,'w')
    fout.write("float32_t R_mock_SAM[%d] = {"%N)
    for k in range(N-1):
        fout.write("%6.5f,"%(Q[k]));
    fout.write("%6.5f};\n"%(Q[N-1]))
    fout.close()
else:
    L = open('mock_L_data_SAM.c','r').readline()
    Lvals = L.split('=')[1].strip().strip(';{}').split(',')
    si = np.array([float(a) for a in Lvals])
    
    R = open('mock_R_data_SAM.c','r').readline()
    Rvals = R.split('=')[1].strip().strip(';{}').split(',')
    sq = np.array([float(a) for a in Rvals])


data = si + 1j*sq
f,Pxx = welch(data,fs=fs,nperseg=2048,return_onesided=False)
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(f/1000,10*np.log10(Pxx),'k.')
plt.xlabel('Frequency [kHz]')
plt.ylabel('Power [dB]')
plt.grid()

plt.subplot(1,2,2)
plt.plot(np.fft.fftshift(f)/1000,10*np.log10(np.fft.fftshift(Pxx)),'k-')
plt.xlabel('Frequency [kHz]')
plt.ylabel('Power [dB]')
plt.grid()
plt.xlim([-1,1])


In [ ]:
SAM_carrier_freq_offsetOld = 0
phzerror = 0.0
fil_out = 0.0
omega2 = 0.0
dc = 0.0
dc_insert = 0.0
dcu = 0.0
dc_insertu = 0.0

def AMDecodeSAM(I,Q):
    global SAM_carrier_freq_offsetOld
    global phzerror
    #global det
    global fil_out
    #global del_out
    global omega2
    global dc
    global dc_insert
    global dcu
    global dc_insertu
    
    TPI = 2*np.pi
    det = 0.0
    del_out = 0.0
    
    pll_fmax = +4000.0
    zeta_help = 65
    zeta = zeta_help / 100.0 # PLL step response: smaller, slower response 1.0 - 0.1
    omegaN = 200.0           # PLL bandwidth 50.0 - 1000.0

    omega_min = TPI * -pll_fmax * 1 / 24000
    omega_max = TPI * pll_fmax * 1 / 24000
    g1 = 1.0 - np.exp(-2.0 * omegaN * zeta * 1 / 24000)
    g2 = -g1 + 2.0 * (1 - np.exp(-omegaN * zeta * 1 / 24000) * np.cos(omegaN * 1 / 24000 * np.sqrt(1.0 - zeta * zeta)))
   

    # fade leveler  // AFP 11-03-22
    tauR = 0.02
    tauI = 1.4
    mtauR = np.exp(-1 / 24000 * tauR)
    onem_mtauR = 1.0 - mtauR
    mtauI = np.exp(-1 / 24000 * tauI)
    onem_mtauI = 1.0 - mtauI
    fade_leveler = 0
    
    # Now, test the synchronous demod algorithm
    corr = np.zeros(2)
    
    
    for i in range(256):
        Sin = np.sin(phzerror)
        Cos = np.cos(phzerror)

        ai = Cos * I[i]
        bi = Sin * I[i]
        aq = Cos * Q[i]
        bq = Sin * Q[i]

        corr[0] = +ai + bq
        corr[1] = -bi + aq

        audio = (ai - bi) + (aq + bq)

        if (fade_leveler):
            dc = mtauR * dc + onem_mtauR * audio
            dc_insert = mtauI * dc_insert + onem_mtauI * corr[0]
            audio = audio + dc_insert - dc
    
        I[i] = audio;
        #if (fade_leveler):
        #    dcu = mtauR * dcu + onem_mtauR * audiou
        #    dc_insertu = mtauI * dc_insertu + onem_mtauI * corr[0]
        #    audiou = audiou + dc_insertu - dcu
        #Q[i] = audiou;
        det = np.arctan2(corr[1], corr[0])

        del_out = fil_out
        omega2 = omega2 + g2 * det
        if (omega2 < omega_min):
            omega2 = omega_min
        else:
            if (omega2 > omega_max):
                omega2 = omega_max
        fil_out = g1 * det + omega2
        phzerror = phzerror + del_out

        # wrap round 2PI, modulus
        while (phzerror >= TPI):
            phzerror -= TPI
        while (phzerror < 0.0):
            phzerror += TPI
    SAM_carrier =  (omega2 * 24000) / (2 * TPI)
    SAM_carrier_freq_offset = 10 * SAM_carrier
    SAM_carrier_freq_offset = 0.95 * SAM_carrier_freq_offsetOld + 0.05 * SAM_carrier_freq_offset
    SAM_carrier_freq_offsetOld = SAM_carrier_freq_offset
    print("%5.4f,%5.4f"%(SAM_carrier , SAM_carrier_freq_offset))

for k in range(int(N/256)):
    AMDecodeSAM(I[k*256:(k+1)*256], Q[k*256:(k+1)*256])
